In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dropout,Flatten, Dense
from keras import applications
from keras.utils.np_utils import to_categorical
import operator
import matplotlib.pyplot as plt
import numpy as np
from math import ceil
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
#Tweakable
epochs = 5
batch_size = 16

#Constants
target_img_width, target_img_height = 224, 224
train_dir = '/dataset/split_data/train/'
val_dir = '/dataset/split_data/validation/'
test_dir = '/dataset/split_data/test/'
vgg_train_features_file = "res/vgg_train_features.npy"
vgg_val_features_file = "res/vgg_val_features.npy"
vgg_test_features_file = "res/vgg_test_features.npy"
num_classes = 10
top_model_weights_path = 'res/_top_model_weights.h5'